In [23]:
import json
import requests
import pandas as pd

In [2]:
!ls ../json


ascarlatti.json  dvorak.json	   meyerbeer.json	schubert.json
bartok.json	 elgar.json	   monteverdi.json	schumann.json
beethoven.json	 faure.json	   morley.json		schutz.json
bellini.json	 franck.json	   mozart.json		scriabin.json
berg.json	 frescobaldi.json  mussorgsky.json	sibelius.json
berlioz.json	 gabrieli.json	   ockeghem.json	smetana.json
binchois.json	 gesualdo.json	   offenbach.json	stamitz.json
bizet.json	 gibbons.json	   paganini.json	stravinsky.json
borodin.json	 gluck.json	   palestrina.json	tallis.json
brahms.json	 gounod.json	   pergolesi.json	taverner.json
bruckner.json	 grieg.json	   perotin.json		tchaikovsky.json
byrd.json	 handel.json	   praetorius.json	telemann.json
caccini.json	 haydn.json	   prokofiev.json	varese.json
chopin.json	 hindemith.json    puccini.json		vaughanwilliams.json
corelli.json	 holst.json	   purcell.json		verdi.json
couperin.json	 ives.json	   rachmaninoff.json	victoria.json
cpebach.json	 josquin.json	   rameau.json		vivaldi.json
debussy.json	 jsb

In [5]:
with open("../json/beethoven.json", "r") as file:
    data = json.load(file)

In [8]:
composer_df = pd.DataFrame(data)
composer_df.iloc[0].scores

['https://imslp.org/wiki/Special:ImagefromIndex/74038',
 'https://imslp.org/wiki/Special:ImagefromIndex/319015']

In [19]:
composer_df['score_ids'] = composer_df.scores.apply(lambda score_lst: [int(x.split("/")[-1]) for x in score_lst])

composer_df.iloc[0].to_dict()

{'composer': 'Beethoven, Ludwig van',
 'date': '1820',
 'scores': ['https://imslp.org/wiki/Special:ImagefromIndex/74038',
  'https://imslp.org/wiki/Special:ImagefromIndex/319015'],
 'tags': ['Canons', 'For 3 voices', 'For unaccompanied voices'],
 'title': 'Abbé Stadler, WoO 178',
 'score_ids': [74038, 319015]}

In [29]:
content = requests.get("https://imslp.org/wiki/Special:IMSLPDisclaimerAccept/51737", headers={"X-Forwarded-For": "24.18.100.31"}).content

with open("../index.html", "w") as file:
    file.write(content.decode("utf-8"))

In [34]:
requests.get("http://127.0.0.1:8080", headers={"X-Forwarded-For": "24.18.100.31"})

<Response [200]>

In [22]:
!curl --header "X-Forwarded-For: 192.168.0.99" https://imslp.org/wiki/Special:ImagefromIndex/51737 > ../index.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22721    0 22721    0     0  92738      0 --:--:-- --:--:-- --:--:-- 92738
